# Model building and training

This notebook is for building and training the sequential keras model on the pre-processed dataset.

The model is a sequential model with two convolutional layers, followed by two dense layers. The hyperparameters are the number of filters, kernel size, and dropout rate. The output layer has two units, one for each class.

In [1]:
import pandas as pd
import os
import librosa
import librosa.display
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization

## Loading preprocessed dataset

In [ ]:
# Load data from pandas dataframe
data = pd.read_csv('audio_features.csv')

# Split data into features and labels
X = data.iloc[:, 2:]
y = data.iloc[:, 1]

# Convert labels to categorical values
y = pd.get_dummies(y)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the features into 2D arrays
X_train = np.array(X_train).reshape(X_train.shape[0], 40, 1)
X_test = np.array(X_test).reshape(X_test.shape[0], 40, 1)

## Build Sequential Model

In [ ]:
# Define the hyperparameters
batch_size = 32
epochs = 20
num_filters = 64
kernel_size = (3,3)
pool_size = (2,2)
dropout_rate = 0.5
dense_units = 64

# Build the model
model = Sequential()
model.add(Conv2D(num_filters, kernel_size, activation='relu', input_shape=(40, 1, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(dropout_rate))

model.add(Conv2D(num_filters, kernel_size, activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(dropout_rate))

model.add(Flatten())

model.add(Dense(dense_units, activation='relu'))
model.add(Dropout(dropout_rate))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Train model

In [ ]:
# Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))

## Save trained model and predictions on test set

In [ ]:
# Save the model
model.save('audio_model.h5')

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)

print("Test set loss: {:.2f}".format(loss))
print("Test set accuracy: {:.2f}%".format(accuracy * 100))